In [ ]:
from langchain_community.utilities import SQLDatabase
from langchain_core.prompts import PromptTemplate
# from langchain.chains import create_sql_query_chain
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI


In [2]:
#connecting mysql database
# host='Ghostriderr323'
# port='3306'
# username=
# password=
# database_schema='text_to_sql'

from dotenv import load_dotenv
import os
load_dotenv()

host=os.getenv("DB_HOST")
port = os.getenv("DB_PORT")
username = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
database_schema = os.getenv("DB_NAME")

mysql_uri=f"mysql+pymysql://{username}:{password}@{host}:{port}/{database_schema}"

db=SQLDatabase.from_uri(mysql_uri,sample_rows_in_table_info=1)


In [3]:
db.get_table_info()

'\nCREATE TABLE `2017_budgets` (\n\t`Product Name` TEXT, \n\t`2017 Budgets` DOUBLE\n)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci\n\n/*\n1 rows from 2017_budgets table:\nProduct Name\t2017 Budgets\nProduct 1\t3016489.2089999998\n*/\n\n\nCREATE TABLE customers (\n\t`Customer Index` INTEGER, \n\t`Customer Names` TEXT\n)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci\n\n/*\n1 rows from customers table:\nCustomer Index\tCustomer Names\n1\tGeiss Company\n*/\n\n\nCREATE TABLE products (\n\t`Index` INTEGER, \n\t`Product Name` TEXT\n)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci\n\n/*\n1 rows from products table:\nIndex\tProduct Name\n1\tProduct 1\n*/\n\n\nCREATE TABLE regions (\n\tid INTEGER, \n\tname TEXT, \n\tcounty TEXT, \n\tstate_code TEXT, \n\tstate TEXT, \n\ttype TEXT, \n\tlatitude DOUBLE, \n\tlongitude DOUBLE, \n\tarea_code INTEGER, \n\tpopulation INTEGER, \n\thouseholds INTEGER, \n\tmedian_income INTEGER, \n\tland_area INTEGER, \

In [4]:
# create a llm prompt template
from langchain_core.prompts import ChatPromptTemplate
template = """
Based on the table schema below, write a SQL query that would answer the user's question.
Remember: Only provide the SQL query, don't include anything else. Provide the SQL query in a single line, don't add line breaks.

Table Schema:{schema}

Question:{question}

SQL Query:
"""

prompt = ChatPromptTemplate.from_template(template)

In [5]:
# get the schema of the database
def get_schema(db):
    schema=db.get_table_info()
    return schema

In [38]:
# llm = ChatGoogleGenerativeAI(
#     model='gemini-2.0-flash',api_key=""
# )
from dotenv import load_dotenv
import os
load_dotenv()

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-3-flash-preview",
    google_api_key=os.getenv("GOOGLE_API_KEY"),
    temperature=0
)


In [ ]:
# create a sql chain using the llm and the prompt template

sql_chain=(
    RunnablePassthrough.assign(schema=lambda _:get_schema(db))
                               | prompt
                               | llm.bind(stop=['\nSQLResult'])
                               | StrOutputParser())

In [40]:
# test the sql query chain with a sample question

response= sql_chain.invoke({"question":"waht is the total 'Line Total' for Geiss Company"})
print(response)


SELECT SUM(T2.`Line Total`) FROM customers AS T1 JOIN sales_order AS T2 ON T1.`Customer Index` = T2.`Customer Name Index` WHERE T1.`Customer Names` = 'Geiss Company'


result of above query

 SELECT SUM(T2.`Line Total`) FROM customers AS T1 JOIN sales_order AS T2 ON T1.`Customer Index` = T2.`Customer Name Index` WHERE T1.`Customer Names` = 'Geiss Company'

In [42]:
#Executing this SQL on the database and return the numeric result
result = db.run(response)
print(result)

[(5516846.999999994,)]


In [52]:
import ast

raw_result = db.run(response)
parsed_res=ast.literal_eval(raw_result)
# value = raw_result[0][0] if raw_result else 0
value = round(float(parsed_res[0][0]),2)

print(value)


5516847.0


In [53]:
# implementing post-sql rag (insight generation)

from langchain_core.prompts import PromptTemplate

answer_prompt=ChatPromptTemplate.from_template("""
                                               user question:{question}
                                               sql result: {result}
                                               
                                               Generate a clear , buisness-friendly answer
                                               """)


In [54]:
answer_chain=(
    answer_prompt
    | llm
    | StrOutputParser()
)

In [57]:
final_answer=answer_chain.invoke(
    {
        "question":"what is the total Line Total for Geiss Company?",
        "result": value
    }
)
print(final_answer)

The total Line Total for Geiss Company is **5,516,847.00**.


In [ ]:
#combining everythin in one pipeline

import ast

def ask_database(question):
    sql= sql_chain.invoke({"question":question})
    result=db.run(sql)
    
    parsed_res=ast.literal_eval(result)
    value = round(float(parsed_res[0][0]),2)
    
    answer= answer_chain.invoke({
        "question":question,
        "result": value
    })
    
    return {
        "sql":sql,
        "value": value,
        "answer": answer
    }

In [61]:
ask_database("what is the total Line Total for Geiss Company?")

{'sql': "SELECT SUM(T2.`Line Total`) FROM customers AS T1 JOIN sales_order AS T2 ON T1.`Customer Index` = T2.`Customer Name Index` WHERE T1.`Customer Names` = 'Geiss Company'",
 'value': 5516847.0,
 'answer': 'The total Line Total for Geiss Company is **5,516,847.00**.'}

In [62]:
#RAGAS IMPLEMENTATION

from datasets import Dataset
eval_sample=[{
    "question": "What is the total Line Total for Geiss Company?",
    "contexts": ["5516847.00"],   # DB result = retrieved context
    "answer": "The total Line Total for Geiss Company is 5,516,847.00.",
    "ground_truth": "The total Line Total for Geiss Company is 5,516,847.00."
}]

dataset=Dataset.from_list(eval_sample)

In [66]:
from ragas.metrics import(
    faithfulness,
    answer_relevancy,
    context_precision
)

C:\Users\ASHISH RAJ\AppData\Local\Temp\ipykernel_632\1330372599.py:1: DeprecationWarning: Importing faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import faithfulness
  from ragas.metrics import(
C:\Users\ASHISH RAJ\AppData\Local\Temp\ipykernel_632\1330372599.py:1: DeprecationWarning: Importing answer_relevancy from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import answer_relevancy
  from ragas.metrics import(
C:\Users\ASHISH RAJ\AppData\Local\Temp\ipykernel_632\1330372599.py:1: DeprecationWarning: Importing context_precision from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import context_precision
  from ragas.metrics import(


In [67]:
from ragas import evaluate

result= evaluate(
    dataset,
    metrics=[
        faithfulness,
        answer_relevancy,
        context_precision
    ]
)
print(result)

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]Exception raised in Job[0]: InstructorRetryException(<failed_attempts>

<generation number="1">
<exception>
    Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
</exception>
<completion>
    None
</completion>
</generation>

<generation number="2">
<exception>
    Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
</exception>
<completion>
    None
</completion>
</generation>

<generation number="3">
<exception>
    Error cod

{'faithfulness': nan, 'answer_relevancy': nan, 'context_precision': nan}


### RAGAS Evaluation
RAGAS was integrated to evaluate faithfulness, answer relevance, and context precision.
Due to evaluator LLM quota limitations on the free tier, metric execution may return NaN.


In [ ]:
from ragas.llms import LangchainLLMWrapper
from langchain_google_genai import ChatGoogleGenerativeAI

evaluator_llm = LangchainLLMWrapper(
    ChatGoogleGenerativeAI(
        model="gemini-3-flash-preview",
        google_api_key=os.getenv("GOOGLE_API_KEY"),
        temperature=0
    )
)

result = evaluate(
    dataset,
    metrics=[faithfulness, answer_relevancy, context_precision],
    llm=evaluator_llm
)

C:\Users\ASHISH RAJ\AppData\Local\Temp\ipykernel_632\3305640405.py:4: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  evaluator_llm = LangchainLLMWrapper(
Evaluating:  33%|███▎      | 1/3 [00:06<00:12,  6.25s/it]LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
Exception raised in Job[1]: RateLimitError(Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}})
Evaluating: 100%|██████████| 3/3 [00:59<00:00, 19.93s/it]
